# Import data and save to csv

## Jason Kniss
### Feb 14 2024

This script is intended to save computation time while editing by saving pertinent data into a csv rather than reload and organize the data with every run.

### Import Libraries

In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
# from IPython.display import HTML
from pandasgui import show

In [2]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

## Import and organize data

### Set path to data folders

separate folders are used because each data source will be treated separately. This is to simplify identification and concatenation.

In [3]:
towermet_path = 'data/eureka-data/tower-met'
# towerrad_path = 'data/eureka-data/tower-rad'
bsrnrad_path = 'data/eureka-data/bsrn-rad'

### Crerate lists of dataframes

Loops will generate a list of dataframes from text files added to specified folders.

In [4]:
towermet_dfs = []  # List to store Tower Meteroological dataframes
# towerrad_dfs = []  # List to store Tower Radiation dataframes
bsrnrad_dfs = [] # List to store BSRN radiation dataframes

### Tower meteorological data

In [5]:
for filename in os.listdir(towermet_path):
    if filename.endswith(".txt"):  # Only consider .txt files
        file_path = os.path.join(towermet_path, filename)
        with open(file_path, "r") as f:
            lines = f.readlines()

        header = lines[0].strip().split()
        data_rows = [line.strip().split("\t") for line in lines[1:]]

        df = pd.DataFrame(data_rows, columns=header)
        # Rename the dataframe when more data is imported

        df = df.astype('float')

        # Fix midnight HourMin
        condition = df['HourMin'] == 2400
        df.loc[condition, 'JulianDay'] += 1
        df.loc[condition, 'HourMin'] = 0000

        condition_60 = df['HourMin'] % 100 == 60
        # Step 2 & 3: Adjust the HourMin values
        # Extract hours and minutes, adjust, and then combine back
        hours = df.loc[condition_60, 'HourMin'] // 100
        minutes = 0  # Reset minutes to 00

        # Increment the hour, and handle the case where it goes beyond 23
        new_hours = (hours + 1) % 24
        df.loc[condition_60, 'HourMin'] = new_hours * 100 + minutes
    
        # Handle the day increment for the case where hours rolled over from 23 to 00
        day_rollover_condition = condition_60 & (hours == 23)
        df.loc[day_rollover_condition, 'JulianDay'] += 1

        df.replace([-999, -9999], np.nan, inplace=True)
        
        towermet_dfs.append(df)

### Tower radiation data

In [6]:
# for filename in os.listdir(towerrad_path):
#     if filename.endswith(".txt"):  # Only consider .txt files
#         file_path = os.path.join(towerrad_path, filename)
#         with open(file_path, "r") as f:
#             lines = f.readlines()

#         header = lines[0].strip().split()
#         data_rows = [line.strip().split("\t") for line in lines[1:]]

#         df = pd.DataFrame(data_rows, columns=header)
#         # Rename the dataframe when more data is imported

#         df = df.astype('float')
        
#         condition = df['HourMin'] == 2400
#         df.loc[condition, 'JulianDay'] += 1
#         df.loc[condition, 'HourMin'] = 0000

#         df.replace([-999, -9999], np.nan, inplace=True)
        
#         #df = df[df['HourMin'] != 2400] # Filter out invalid HourMin values 
#         towerrad_dfs.append(df)

### BSRN radiation data
This is redundant of tower radiation data and only serves as a comparison to tower measurements

In [7]:
for filename in os.listdir(bsrnrad_path):
    if filename.endswith(".txt"):  # Only consider .txt files
        file_path = os.path.join(bsrnrad_path, filename)
        with open(file_path, "r") as f:
            lines = f.readlines()

        header = lines[0].strip().split()
        data_rows = [line.strip().split("\t") for line in lines[1:]]

        df = pd.DataFrame(data_rows, columns=header)
        # Rename the dataframe when more data is imported

        df = df.astype('float')
        
        condition = df['HourMin'] == 2400
        df.loc[condition, 'JulianDay'] += 1
        df.loc[condition, 'HourMin'] = 0000

 
        condition_60 = df['HourMin'] % 100 == 60
        # Step 2 & 3: Adjust the HourMin values
        # Extract hours and minutes, adjust, and then combine back
        hours = df.loc[condition_60, 'HourMin'] // 100
        minutes = 0  # Reset minutes to 00

        # Increment the hour, and handle the case where it goes beyond 23
        new_hours = (hours + 1) % 24
        df.loc[condition_60, 'HourMin'] = new_hours * 100 + minutes
    
        # Handle the day increment for the case where hours rolled over from 23 to 00
        day_rollover_condition = condition_60 & (hours == 23)
        df.loc[day_rollover_condition, 'JulianDay'] += 1

        df.replace([-999, -9999], np.nan, inplace=True)
        
        #df = df[df['HourMin'] != 2400] # Filter out invalid HourMin values 
        bsrnrad_dfs.append(df)

## Sort and concatenate lists of dataframes

Done separately to simplify concatenation in the next step
### Create datetime columns
### Tower meteorological dataframes

In [8]:
# Create a Datetime column 
for df in towermet_dfs:
    def julian_to_date(julian_day, base_year=(df['Year'].iloc[0]).astype('int')):
        base_date = datetime(base_year, 1, 1)
        target_date = base_date + timedelta(days=julian_day - 1)
        return target_date.strftime('%Y-%m-%d')

    def hourmin_to_time(hourmin):
        hours = int(hourmin // 100)
        minutes = int(hourmin % 100)
        return f'{hours:02}:{minutes:02}:00'

    # Convert Julian Day to date
    df['Date'] = df['JulianDay'].apply(julian_to_date)

    # Convert HourMin to time   
    df['Time'] = df['HourMin'].apply(hourmin_to_time)

    # Combine date and time into a single datetime column
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

    # Drop intermediate columns
    df.drop(columns=['Date', 'Time'], inplace=True)
    


### Tower radiation dataframes

In [9]:
# for df in towerrad_dfs:
#       def julian_to_date(julian_day, base_year=(df['Year'].iloc[0]).astype('int')):
#           base_date = datetime(base_year, 1, 1)
#           target_date = base_date + timedelta(days=julian_day - 1)
#           return target_date.strftime('%Y-%m-%d')

#       def hourmin_to_time(hourmin):
#           hours = int(hourmin // 100)
#           minutes = int(hourmin % 100)
#           return f'{hours:02}:{minutes:02}:00'

#       # Convert Julian Day to date
#       df['Date'] = df['JulianDay'].apply(julian_to_date)

#       # Convert HourMin to time   
#       df['Time'] = df['HourMin'].apply(hourmin_to_time)

#       # Combine date and time into a single datetime column
#       df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

#       # Drop intermediate columns
#       df.drop(columns=['Date', 'Time'], inplace=True)
      
#       # Set 'Datetime' column as the index
#  #     df.set_index('Datetime', inplace=True)

### BSRN radiation dataframes

In [10]:
for df in bsrnrad_dfs:
    def julian_to_date(julian_day, base_year=(df['Year'].iloc[0]).astype('int')):
        base_date = datetime(base_year, 1, 1)
        target_date = base_date + timedelta(days=julian_day - 1)
        return target_date.strftime('%Y-%m-%d')

    def hourmin_to_time(hourmin):
        hours = int(hourmin // 100)
        minutes = int(hourmin % 100)
        return f'{hours:02}:{minutes:02}:00'

    # Convert Julian Day to date
    df['Date'] = df['JulianDay'].apply(julian_to_date)

    # Convert HourMin to time   
    df['Time'] = df['HourMin'].apply(hourmin_to_time)

    # Combine date and time into a single datetime column
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

    # Drop intermediate columns
    df.drop(columns=['Date', 'Time'], inplace=True)

### Concatenate lists of dataframes into dataframes and reindex by datetime

In [11]:
# Concatenate vertically
towermet_df = pd.concat(towermet_dfs, ignore_index=True)
# towerrad_df = pd.concat(towerrad_dfs, ignore_index=True)
bsrnrad_df = pd.concat(bsrnrad_dfs, ignore_index=True)

In [12]:
# Sort by datetime column
towermet_df.sort_values(by='Datetime', inplace=True)    
# towerrad_df.sort_values(by='Datetime', inplace=True)  
bsrnrad_df.sort_values(by='Datetime', inplace=True)

In [14]:
duplicates = towermet_df['Datetime'].duplicated(keep=False)

# Iterate over the DataFrame, and for each duplicate datetime, increment the second occurrence by one minute
for i, is_duplicate in enumerate(duplicates):
    if is_duplicate:
        # If this row's datetime is the same as the next row's datetime, it's the first occurrence
        if i+1 < len(towermet_df) and towermet_df.iloc[i]['Datetime'] == towermet_df.iloc[i+1]['Datetime']:
            continue
        # Otherwise, it's the second occurrence, so increment by one minute
        else:
            towermet_df.at[i, 'Datetime'] += pd.Timedelta(minutes=1)


In [15]:
# Set 'Datetime' column as the index
towermet_df.set_index('Datetime', inplace=True)
# towerrad_df.set_index('Datetime', inplace=True)
bsrnrad_df.set_index('Datetime', inplace=True)

### Join dataframes into one dataframe with pertinant columns

In [17]:
# Tower Met Data
towermet_df_select = towermet_df[[
    'Pressure[mbar]',
    '10MRH[%]',
    # '6MRH[%]',
    '2MRH[%]',
    '10MVTair[degC]',
    # '6MVTair[degC]',
    '2MVTair[degC]'
]]

# Tower Radiation Data
# towerrad_df_select = towerrad_df[[
#     'LWTotalDownwelling[W/m^2]',
#     'QualityControl'
# ]].copy()

# towerrad_df_select.rename(columns={'LWTotalDownwelling[W/m^2]': 'twrLWTotalDownwelling[W/m^2]'}, inplace=True)
# towerrad_df_select.rename(columns={'QualityControl': 'twrQualityControl'}, inplace=True)

# # BSRN Radiation Data
bsrnrad_df_select = bsrnrad_df[[
    'LWTotalDownwelling[W/m^2]',
    'QualityControl'
]].copy()

bsrnrad_df_select.rename(columns={'LWTotalDownwelling[W/m^2]': 'bsrnLWTotalDownwelling[W/m^2]'}, inplace=True)
bsrnrad_df_select.rename(columns={'QualityControl': 'bsrnQualityControl'}, inplace=True)

# Join Dataframes
data_df = towermet_df_select.join(bsrnrad_df_select, how='outer') #.join(towerrad_df_select, how='outer')

In [19]:
data_df.describe()

,Pressure[mbar],10MRH[%],2MRH[%],10MVTair[degC],2MVTair[degC],bsrnLWTotalDownwelling[W/m^2],bsrnQualityControl
count,502711.000000,502709.000000,502709.000000,502709.000000,502709.000000,576723.000000,5.769580e+05
mean,999.752830,74.581420,72.301813,-14.971807,-15.321710,701.924128,8.115233e+17
std,9.463734,12.327847,12.148513,15.987153,16.398681,509.837733,1.843108e+18
min,971.900000,27.363000,26.511000,-42.902000,-44.235000,-63453.000000,6.006044e+14
25%,992.690000,68.430000,65.513000,-29.968000,-30.699000,237.930000,9.091991e+15
50%,1000.500000,75.270000,72.548000,-17.696000,-18.137000,743.930000,9.610694e+15
75%,1006.900000,82.705000,80.887000,-0.586780,-0.589520,1053.200000,9.691994e+15
max,1021.800000,111.540000,102.150000,18.131000,18.624000,3210.100000,9.999992e+18


In [21]:
data_df.columns

Index(['Pressure[mbar]', '10MRH[%]', '2MRH[%]', '10MVTair[degC]',
       '2MVTair[degC]', 'bsrnLWTotalDownwelling[W/m^2]', 'bsrnQualityControl'],
      dtype='object')

In [23]:
# Extract the year from the index
year = data_df.index.year[0]

# Format the filename with the year
filename = f"eureka-data-{year}.csv"

data_df.to_csv(filename)